## Import packages

In [1]:
import math
import pandas as pd
import matplotlib.pyplot as plt
from keras.callbacks import Callback, EarlyStopping, ModelCheckpoint
import sys
sys.path.append("H:/WareHouse/MovieBigData/keras-movielens-cf-master/")

from CFModel import CFModel

Using TensorFlow backend.


## Define constants


In [2]:
RATINGS_CSV_FILE = 'H:/WareHouse/MovieBigData/keras-movielens-cf-master/ml1m_ratings.csv'
MODEL_WEIGHTS_FILE = 'H:/WareHouse/MovieBigData/keras-movielens-cf-master/ml1m_weights.h5'
K_FACTORS = 120
RNG_SEED = 1446557

## Load MovieLens 1M data

In [3]:
ratings = pd.read_csv(RATINGS_CSV_FILE, 
                      sep='\t', 
                      encoding='latin-1', 
                      usecols=['userid', 'movieid', 'user_emb_id', 'movie_emb_id', 'rating'])
max_userid = ratings['userid'].drop_duplicates().max()
max_movieid = ratings['movieid'].drop_duplicates().max()
print(len(ratings), 'ratings loaded.')

1000209 ratings loaded.


## Create training set

In [4]:
shuffled_ratings = ratings.sample(frac=1., random_state=RNG_SEED)
Users = shuffled_ratings['user_emb_id'].values
print ('Users:', Users, ', shape =', Users.shape)
Movies = shuffled_ratings['movie_emb_id'].values
print( 'Movies:', Movies, ', shape =', Movies.shape)
Ratings = shuffled_ratings['rating'].values
print ('Ratings:', Ratings, ', shape =', Ratings.shape)

Users: [4403 1819 2571 ...,  354 3510  831] , shape = (1000209,)
Movies: [2717 3692  149 ..., 2700 2920 3711] , shape = (1000209,)
Ratings: [5 4 4 ..., 2 4 4] , shape = (1000209,)


## Define model

In [5]:
model = CFModel(max_userid, max_movieid, K_FACTORS)
model.compile(loss='mse', optimizer='adamax')

# from keras.utils import plot_modelplot_model(model, to_file='model.png')


H:/WareHouse/MovieBigData/keras-movielens-cf-master\CFModel.py:25: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  self.add(Merge([P, Q], mode='dot', dot_axes=1))


ImportError: Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.

## Train model

In [9]:
callbacks = [EarlyStopping('val_loss', patience=2), 
             ModelCheckpoint(MODEL_WEIGHTS_FILE, save_best_only=True)]
history = model.fit([Users, Movies], Ratings, nb_epoch=30, validation_split=.1, verbose=2, callbacks=callbacks)

H:\Python\Anaconda3\lib\site-packages\keras\models.py:826: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 900188 samples, validate on 100021 samples
Epoch 1/30


KeyboardInterrupt: 

## Plot training and validation RMSE

In [1]:
loss = pd.DataFrame({'epoch': [ i + 1 for i in history.epoch ],
                     'training': [ math.sqrt(loss) for loss in history.history['loss'] ],
                     'validation': [ math.sqrt(loss) for loss in history.history['val_loss'] ]})
ax = loss.ix[:,:].plot(x='epoch', figsize={7,10}, grid=True)
ax.set_ylabel("root mean squared error")
ax.set_ylim([0.0,3.0])

NameError: name 'pd' is not defined

## Print best validation RMSE

In [2]:
min_val_loss, idx = min((val, idx) for (idx, val) in enumerate(history.history['val_loss']))
print ('Minimum RMSE at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(math.sqrt(min_val_loss)))

NameError: name 'history' is not defined